In [4]:
# General imports
import numpy as np
import scipy.io
from sklearn.preprocessing import OneHotEncoder

# Custom imports
from modules import RC_model

In [5]:
# ============ RC model configuration and hyperparameter values ============
config = {}
config['dataset_name'] = 'ESNN5np564Both042621'

config['seed'] = 1
np.random.seed(config['seed'])

# Hyperarameters of the reservoir
config['n_internal_units'] = 495        # size of the reservoir
config['spectral_radius'] = 0.53        # largest eigenvalue of the reservoir
config['leak'] = 0.66                   # amount of leakage in the reservoir state update (None or 1.0 --> no leakage)
config['connectivity'] = 0.25           # percentage of nonzero connections in the reservoir
config['input_scaling'] = 0.0878        # scaling of the input weights
config['noise_level'] = 0.01            # noise in the reservoir state update
config['n_drop'] = 5                    # transient states to be dropped
config['bidir'] = False                 # if True, use bidirectional reservoir
config['circ'] = False                  # use reservoir with circle topology

# Dimensionality reduction hyperparameters
config['dimred_method'] ='pca'       # options: {None (no dimensionality reduction), 'pca', 'tenpca'}
config['n_dim'] = 168                    # number of resulting dimensions after the dimensionality reduction procedure

# Type of MTS representation
config['mts_rep'] = 'reservoir'         # MTS representation:  {'last', 'mean', 'output', 'reservoir'}
config['w_ridge_embedding'] = 11.0      # regularization parameter of the ridge regression

# Type of readout
config['readout_type'] = 'lin'          # readout used for classification: {'lin', 'mlp', 'svm'}

# Linear readout hyperparameters
config['w_ridge'] = 4.0                 # regularization of the ridge regression readout

# SVM readout hyperparameters
config['svm_gamma'] = 0.0055             # bandwith of the RBF kernel
config['svm_C'] = 5.5                   # regularization for SVM hyperplane

# MLP readout hyperparameters
config['mlp_layout'] = (9,9)          # neurons in each MLP layer
config['num_epochs'] = 2000             # number of epochs 
config['w_l2'] = 0.001                  # weight of the L2 regularization
config['nonlinearity'] = 'tanh'         # type of activation function {'relu', 'tanh', 'logistic', 'identity'}

print(config)

{'dataset_name': 'ESNN5np564Both042621', 'seed': 1, 'n_internal_units': 495, 'spectral_radius': 0.53, 'leak': 0.66, 'connectivity': 0.25, 'input_scaling': 0.0878, 'noise_level': 0.01, 'n_drop': 5, 'bidir': False, 'circ': False, 'dimred_method': 'pca', 'n_dim': 168, 'mts_rep': 'reservoir', 'w_ridge_embedding': 11.0, 'readout_type': 'lin', 'w_ridge': 4.0, 'svm_gamma': 0.0055, 'svm_C': 5.5, 'mlp_layout': (9, 9), 'num_epochs': 2000, 'w_l2': 0.001, 'nonlinearity': 'tanh'}


In [6]:
# ============ Load dataset ============
data = scipy.io.loadmat('../dataset/'+config['dataset_name']+'.mat')
Xtr = data['X']  # shape is [N,T,V]
if len(Xtr.shape) < 3:
    Xtr = np.atleast_3d(Xtr)
Ytr = data['Y']  # shape is [N,1]
Xte = data['Xte']
if len(Xte.shape) < 3:
    Xte = np.atleast_3d(Xte)
Yte = data['Yte']

print('Loaded '+config['dataset_name']+' - Tr: '+ str(Xtr.shape)+', Te: '+str(Xte.shape))

# One-hot encoding for labels
onehot_encoder = OneHotEncoder(sparse=False)
Ytr = onehot_encoder.fit_transform(Ytr)
Yte = onehot_encoder.transform(Yte)

Loaded ESNN5np564Both042621 - Tr: (5509, 128, 45), Te: (3649, 128, 45)


In [7]:
# ============ Initialize, train and evaluate the RC model ============
classifier =  RC_model(
                        reservoir=None,     
                        n_internal_units=config['n_internal_units'],
                        spectral_radius=config['spectral_radius'],
                        leak=config['leak'],
                        connectivity=config['connectivity'],
                        input_scaling=config['input_scaling'],
                        noise_level=config['noise_level'],
                        circle=config['circ'],
                        n_drop=config['n_drop'],
                        bidir=config['bidir'],
                        dimred_method=config['dimred_method'], 
                        n_dim=config['n_dim'],
                        mts_rep=config['mts_rep'],
                        w_ridge_embedding=config['w_ridge_embedding'],
                        readout_type=config['readout_type'],            
                        w_ridge=config['w_ridge'],              
                        mlp_layout=config['mlp_layout'],
                        num_epochs=config['num_epochs'],
                        w_l2=config['w_l2'],
                        nonlinearity=config['nonlinearity'], 
                        svm_gamma=config['svm_gamma'],
                        svm_C=config['svm_C']
                        )

tr_time = classifier.train(Xtr, Ytr)
print('Training time = %.2f seconds'%tr_time)

accuracy, f1 = classifier.test(Xte, Yte)
print('Accuracy = %.3f, F1 = %.3f'%(accuracy, f1))


Training time = 1.64 seconds
Accuracy = 0.814, F1 = 0.809


In [ ]:
classifier.predictions(Xte, Yte)